In [2]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

In [4]:
df = pd.read_csv("data_sets/API_UGA_DS2_en_csv_v2_28828.csv")
#df

In [5]:
df1 = pd.read_csv("data_sets/API_UGA_DS2_en_csv_v2_28828.csv", usecols=[2] + list(range(19, 27)))
df1 = df1.dropna()
#df1 = df1.round(2)
df1

,Indicator Name,1975,1976,1977,1978,1979,1980,1981,1982
5,Merchandise exports by the reporting economy (...,278130000.00,360553000.00,564932000.00,296561000.00,277656000.00,163020000.00,265342936.30,356370000.00
6,Merchandise exports to low- and middle-income ...,3.86,1.25,5.36,8.45,2.71,5.04,2.92,3.97
7,Merchandise exports to low- and middle-income ...,6.32,2.31,8.16,9.44,3.77,5.04,3.35,4.55
12,Merchandise imports from low- and middle-incom...,34.41,50.48,57.43,28.86,33.61,30.94,42.41,33.55
13,Merchandise imports from low- and middle-incom...,3.33,1.90,3.02,9.43,0.94,12.16,7.95,9.12
...,...,...,...,...,...,...,...,...,...
1489,Cereal yield (kg per hectare),1350.80,1217.20,1205.40,1210.10,1613.70,1491.00,1561.70,1309.00
1490,Crop production index (2014-2016 = 100),94.61,86.48,86.60,85.95,60.11,59.31,63.86,71.15
1491,Land area (sq. km),199810.00,199810.00,199810.00,199810.00,199810.00,199810.00,199810.00,199810.00
1494,Arable land (% of land area),20.14,20.38,20.13,20.42,20.42,20.42,20.62,20.92


In [10]:
#df2 = df1.T
#df2.describe()
#df2 = df1.iloc[:, [0]]
#df2.to_html('df2_output.html')

In [11]:
n = len(df1.iloc[:, 0])
df1 = df1.reset_index(drop=True)
rows_to_drop = []

for el in range(0, n):
    if df1.iloc[el, 3] == df1.iloc[el, 4]:
        rows_to_drop.append(el)

df1 = df1.drop(rows_to_drop)
#df1

In [12]:
#df1.to_html('df1_output.html')
#df2 = df1.iloc[:, [0]] #kolumna
#df2 = df1.iloc[[0], :] #wiersz
#df2

In [13]:
#a = df1.iloc[[249], :]
#a

In [14]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import f

imp = []
n = len(df1.iloc[:, 0])

for el in range(0, n):
    dt = df1.iloc[el, :].tolist()
    dt.pop(0)
    
    # Przykładowe dane
    data = pd.DataFrame({
    'X': np.arange(1, len(dt)+1),
    'Y': dt
    })
    
    # Punkt podziału (np. zmiana władzy)
    split_point = 4
    
    # Dzielimy dane na dwa okresy: przed i po zmianie
    data_before = data[:split_point]
    data_after = data[split_point:]
    
    # Funkcja do obliczania sumy kwadratów reszt (RSS)
    def rss(y, y_pred):
        return np.sum((y - y_pred) ** 2)
    
    # Model pełny (bez podziału)
    X_full = sm.add_constant(data['X'])
    model_full = sm.OLS(data['Y'], X_full).fit()
    rss_full = rss(data['Y'], model_full.predict(X_full))
    
    # Model dla okresu przed podziałem
    X_before = sm.add_constant(data_before['X'])
    model_before = sm.OLS(data_before['Y'], X_before).fit()
    rss_before = rss(data_before['Y'], model_before.predict(X_before))
    
    # Model dla okresu po podziale
    X_after = sm.add_constant(data_after['X'])
    model_after = sm.OLS(data_after['Y'], X_after).fit()
    rss_after = rss(data_after['Y'], model_after.predict(X_after))
    
    # Liczba parametrów i obserwacji
    k = X_full.shape[1]  # Liczba parametrów (stała i nachylenie)
    n1 = len(data_before)
    n2 = len(data_after)
    
    # Statystyka testu Chow
    numerator = (rss_full - (rss_before + rss_after)) / k
    denominator = (rss_before + rss_after) / (n1 + n2 - 2 * k)
    chow_stat = numerator / denominator
    
    # Stopnie swobody
    df_numerator = k
    df_denominator = (n1 + n2 - 2 * k)
    
    # Obliczenie p-wartości
    p_value = 1 - f.cdf(chow_stat, df_numerator, df_denominator)
    
    if p_value > 0.05:
        imp.append(el)

df3 = pd.DataFrame(columns=df1.columns)

for el in imp:
    df3.loc[len(df3)] = df1.iloc[el]

df3.to_html('df3_output.html')
df3

,Indicator Name,1975,1976,1977,1978,1979,1980,1981,1982
0,Merchandise exports by the reporting economy (...,278130000.00,360553000.00,564932000.00,296561000.00,277656000.00,163020000.00,265342936.30,356370000.00
1,Merchandise exports to low- and middle-income ...,3.86,1.25,5.36,8.45,2.71,5.04,2.92,3.97
2,Merchandise exports to low- and middle-income ...,6.32,2.31,8.16,9.44,3.77,5.04,3.35,4.55
3,Merchandise imports from low- and middle-incom...,34.41,50.48,57.43,28.86,33.61,30.94,42.41,33.55
4,Merchandise imports from low- and middle-incom...,3.33,1.90,3.02,9.43,0.94,12.16,7.95,9.12
...,...,...,...,...,...,...,...,...,...
176,Net official development assistance and offici...,205389999.39,82440002.44,63259998.32,123519996.64,134119995.12,289859985.35,360079986.57,356019989.01
177,"Net official flows from UN agencies, WFP (curr...",400000.01,470000.00,670000.02,1610000.01,3299999.95,4909999.85,12659999.85,9659999.85
178,"Grants, excluding technical cooperation (BoP, ...",33940000.00,4790000.00,6100000.00,5310000.00,35740000.00,61420000.00,100430000.00,55630000.00
179,Arable land (% of land area),20.14,20.38,20.13,20.42,20.42,20.42,20.62,20.92


In [16]:
from scipy.stats import ttest_rel
imp = []
n = len(df1.iloc[:, 0])
for el in range(0, n):
    all = df1.iloc[el].tolist()
    all.pop(0)
    before = all[0:3]
    after = all[4:7]
    
    stat, p_value = ttest_rel(before, after)
    
    if p_value < 0.05:
        imp.append(el)

df2 = pd.DataFrame(columns=df1.columns)

for el in imp:
    df2.loc[len(df2)] = df1.iloc[el]

df2.to_html('df2_output.html')
df2

C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_axis_nan_policy.py:573: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


,Indicator Name,1975,1976,1977,1978,1979,1980,1981,1982
0,"Population, female (% of total population)",50.45,50.50,50.55,50.60,50.67,50.74,50.83,50.92
1,Age dependency ratio (% of working-age populat...,101.34,101.90,102.31,102.56,103.07,103.86,104.40,104.56
2,"Population ages 75-79, male (% of male populat...",0.41,0.41,0.42,0.42,0.43,0.43,0.43,0.44
3,Population ages 65 and above (% of total popul...,2.68,2.72,2.76,2.80,2.83,2.86,2.88,2.90
4,"Population ages 65 and above, female (% of fem...",2.62,2.68,2.74,2.79,2.84,2.89,2.93,2.97
...,...,...,...,...,...,...,...,...,...
128,Methane (CH4) emissions (total) excluding LULU...,11.18,11.54,11.62,12.31,12.61,12.35,12.49,12.68
129,Methane (CH4) emissions from Building (Energy)...,1.00,1.03,1.06,1.11,1.19,1.23,1.25,1.26
130,Cereal yield (kg per hectare),1350.80,1217.20,1205.40,1210.10,1613.70,1491.00,1561.70,1309.00
131,Crop production index (2014-2016 = 100),94.61,86.48,86.60,85.95,60.11,59.31,63.86,71.15


In [11]:
'''
from statsmodels.multivariate.manova import MANOVA
data_long = df1.melt(id_vars='indicator', var_name='rok', value_name='wartosc')
data_wide = data_long.pivot(index='rok', columns='indicator', values='wartosc').reset_index()

data_wide.columns = data_wide.columns.str.replace(r'[^a-zA-Z0-9]', '_', regex=True)

zmienne_zalezne = data_wide.columns.tolist()

zmienne_zalezne.pop(0)

formula = ' + '.join(zmienne_zalezne) + ' ~ rok'

maov = MANOVA.from_formula(formula, data_wide)
'''

"\nfrom statsmodels.multivariate.manova import MANOVA\ndata_long = df1.melt(id_vars='indicator', var_name='rok', value_name='wartosc')\ndata_wide = data_long.pivot(index='rok', columns='indicator', values='wartosc').reset_index()\n\ndata_wide.columns = data_wide.columns.str.replace(r'[^a-zA-Z0-9]', '_', regex=True)\n\nzmienne_zalezne = data_wide.columns.tolist()\n\nzmienne_zalezne.pop(0)\n\nformula = ' + '.join(zmienne_zalezne) + ' ~ rok'\n\nmaov = MANOVA.from_formula(formula, data_wide)\n"

In [13]:
'''
print(data_wide.isnull().sum())

# Usunięcie wierszy z brakującymi wartościami
data_wide = data_wide.dropna()

from sklearn.preprocessing import StandardScaler

# Skalowanie zmiennych zależnych
scaler = StandardScaler()
data_wide[zmienne_zalezne] = scaler.fit_transform(data_wide[zmienne_zalezne])

wyniki = maov.mv_test()
'''

'\nprint(data_wide.isnull().sum())\n\n# Usunięcie wierszy z brakującymi wartościami\ndata_wide = data_wide.dropna()\n\nfrom sklearn.preprocessing import StandardScaler\n\n# Skalowanie zmiennych zależnych\nscaler = StandardScaler()\ndata_wide[zmienne_zalezne] = scaler.fit_transform(data_wide[zmienne_zalezne])\n\nwyniki = maov.mv_test()\n'